<a href="https://colab.research.google.com/github/AleksTurov/Data-Science/blob/main/ETHUSDT/ETHUSDT_futures_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Определение собственные движения цены фьючерса ETHUSDT  


1. Нам нужно определить собственные движения цены фьючерса ETHUSDT, исключив из них движения вызванные влиянием цены BTCUSDT. Опишите, какую методику вы выбрали, какие параметры подобрали, и почему.  

2. Напишите программу на Python, которая в реальном времени (с минимальной задержкой) следит за ценой фьючерса ETHUSDT и используя выбранный вами метод, определяет собственные движение цены ETH. При изменении цены на 1% за последние 60 минут, программа выводит сообщение в консоль. При этом программа должна продолжать работать дальше, постоянно считывая актуальную цену.  


## 1. Методика


Для определения собственных движений цены ETH, исключив из них движения вызванные влиянием цены BTC, можно использовать методику, основанную на корреляции между ценами ETHUSDT и BTCUSDT.  

Одной из возможных методик является вычисление корреляции между ценами ETHUSDT и BTCUSDT на некотором временном интервале (например, за последние 60 минут) и отфильтровывание движений цены ETHUSDT, которые можно объяснить корреляцией с ценой BTCUSDT. Для этого можно использовать формулу корреляции Пирсона:  

## Код

Вот пример кода, который использует описанную методику для определения собственных движений цены ETHUSDT и выводит сообщение в консоль, если цена изменилась на 1% за последние 60 минут:  

In [5]:
!pip install python-binance
import numpy as np
from binance.client import Client
import time


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
import socket   
hostname=socket.gethostname()   
IPAddr=socket.gethostbyname(hostname)   
print("Your Computer Name is:"+hostname)   
print("Your Computer IP Address is:"+IPAddr)   

Your Computer Name is:codespaces-e8d1ee
Your Computer IP Address is:127.0.0.1


In [7]:
# задаем параметры подключения к API Binance
api_key = 'your_api_key'
api_secret = 'your_api_secret'


client = Client(api_key, api_secret)

Установите ваши api key and your_api_secret

In [8]:
# методика определения собственных движений цены ETHUSDT
def own_price_movements():
    # получение исторических данных цен ETHUSDT и BTCUSDT
    eth_prices = np.array(client.futures_historical_klines('ETHUSDT', Client.KLINE_INTERVAL_1MINUTE, '60 min ago UTC'))
    btc_prices = np.array(client.futures_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_1MINUTE, '60 min ago UTC'))

    # вычисление корреляции между ценами ETHUSDT и BTCUSDT
    corr_coef = np.corrcoef(eth_prices[:, 4].astype(float), btc_prices[:, 4].astype(float))[0, 1]

    # фильтрация движений цены ETHUSDT, которые можно объяснить корреляцией с ценой BTCUSDT
    own_eth_prices = eth_prices[:, 4].astype(float) - corr_coef * btc_prices[:, 4].astype(float)

    return own_eth_prices

# задержка между запросами в секундах
delay = 1

# начальная цена фьючерса ETHUSDT
prev_price = float(client.futures_ticker(symbol='ETHUSDT')['lastPrice'])

# главный цикл программы
while True:
    # текущая цена фьючерса ETHUSDT
    curr_price = float(client.futures_ticker(symbol='ETHUSDT')['lastPrice'])

    # вычисление процентного изменения цены за последние 60 минут
    own_prices = own_price_movements()
    own_prices_last_hour = own_prices[-60:]
    own_price_change = (curr_price - own_prices_last_hour[0]) / own_prices_last_hour[0] * 100

    # проверка условия изменения цены на 1% за последние 60 минут
    if abs(own_price_change) >= 1:
        print(f"Price changed by {own_price_change:.2f}% in the last hour: {prev_price:.2f} -> {curr_price:.2f}")

    # сохранение текущей цены фьючерса ETHUSDT как предыдущей
    prev_price = curr_price

    # задержка перед следующим запросом
    time.sleep(delay)


Price changed by -107.95% in the last hour: 1626.22 -> 1626.22
Price changed by -107.95% in the last hour: 1626.22 -> 1626.22
Price changed by -107.95% in the last hour: 1626.22 -> 1626.15
Price changed by -107.93% in the last hour: 1626.15 -> 1626.14
Price changed by -107.93% in the last hour: 1626.14 -> 1626.06
Price changed by -107.93% in the last hour: 1626.06 -> 1625.46
Price changed by -107.93% in the last hour: 1625.46 -> 1625.68
Price changed by -107.93% in the last hour: 1625.68 -> 1625.99
Price changed by -107.94% in the last hour: 1625.99 -> 1625.33
Price changed by -107.92% in the last hour: 1625.33 -> 1625.34
Price changed by -107.93% in the last hour: 1625.34 -> 1625.47
Price changed by -107.93% in the last hour: 1625.47 -> 1625.42
Price changed by -107.93% in the last hour: 1625.42 -> 1625.42
Price changed by -107.93% in the last hour: 1625.42 -> 1625.18
Price changed by -107.93% in the last hour: 1625.18 -> 1625.34
Price changed by -107.93% in the last hour: 1625.34 -> 

KeyboardInterrupt: 

**Обязательно замените your_api_keyи your_api_secretна фактические значения вашего ключа и секрета API.**

В этом примере мы используем метод futures_historical_klines из библиотеки python-binance, чтобы получить исторические данные цен ETHUSDT и BTCUSDT за последние 60 минут с интервалом свечи в 1 минуту. Затем мы вычисляем корреляцию между ценами ETHUSDT и BTCUSDT с помощью функции np.corrcoef из библиотеки NumPy. Наконец, мы вычитаем из цен ETHUSDT произведение коэффициента корреляции и цен BTCUSDT, чтобы получить собственные движения цены ETHUSDT.  

Это только пример методики, и ее можно дополнить или улучшить в зависимости от вашей задачи и данных.   